In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import joblib

#  Part 1: Data Preprocessing


In [2]:

def extract_frames(video_path, output_dir, fps=1):
    os.makedirs(output_dir, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    extracted_frames = []

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if int(cap.get(cv2.CAP_PROP_FPS)) > 0 and frame_count % int(cap.get(cv2.CAP_PROP_FPS) // fps) == 0:
            frame_path = os.path.join(output_dir, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_path, frame)
            extracted_frames.append(frame_path)
        frame_count += 1

    cap.release()
    return extracted_frames

In [3]:

def load_and_preprocess_data(csv_path, image_dir):
    df = pd.read_csv(csv_path)

    structured_features = ['location_type', 'weapon_type', 'time_of_day']
    numeric_features = ['casualties']
    categorical_features = ['location_type', 'weapon_type', 'time_of_day']

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ])

    image_paths = []
    labels = []
    structured_data = []

    for _, row in df.iterrows():
        img_path = os.path.join(image_dir, f"incident_{row['incident_id']}.jpg")
        if os.path.exists(img_path):
            image_paths.append(img_path)
            labels.append(1 if row['suspicious'] else 0)
            structured_row = row[structured_features + numeric_features]
            structured_data.append(structured_row.values)

    structured_data = np.array(structured_data)
    labels = np.array(labels)
    return image_paths, structured_data, labels, preprocessor

In [4]:
def preprocess_image(image_path, target_size=(224, 224)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, target_size)
    img = img / 255.0
    return img

# Part 2: Model Building

In [5]:
def create_cnn_model(input_shape):
    inputs = Input(shape=input_shape)
    x = Conv2D(32, (3, 3), activation='relu')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    return Model(inputs, x, name='cnn_model')

In [6]:
def create_combined_model(image_shape, num_structured_features):
    image_input = Input(shape=image_shape, name='image_input')
    cnn_model = create_cnn_model(image_shape)
    image_output = cnn_model(image_input)

    structured_input = Input(shape=(num_structured_features,), name='structured_input')
    x = Dense(32, activation='relu')(structured_input)
    x = Dropout(0.2)(x)
    structured_output = Dense(16, activation='relu')(x)

    combined = concatenate([image_output, structured_output])
    x = Dense(64, activation='relu')(combined)
    x = Dropout(0.3)(x)
    output = Dense(2, activation='softmax')(x)

    return Model(inputs=[image_input, structured_input], outputs=output)

# Part 3: Training Pipeline

In [7]:
def train_model(image_paths, structured_data, labels, preprocessor):
    images = np.array([preprocess_image(path) for path in image_paths])
    structured_data = preprocessor.fit_transform(structured_data)

    X_train_img, X_test_img, X_train_struct, X_test_struct, y_train, y_test = train_test_split(
        images, structured_data, labels, test_size=0.2, random_state=42
    )

    y_train = to_categorical(y_train, 2)
    y_test = to_categorical(y_test, 2)

    model = create_combined_model(images[0].shape, structured_data.shape[1])
    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(
        [X_train_img, X_train_struct],
        y_train,
        epochs=15,
        batch_size=32,
        validation_data=([X_test_img, X_test_struct], y_test)
    )

    model.save('surveillance_model.h5')
    joblib.dump(preprocessor, 'preprocessor.joblib')
    return model, history

# Part 4: Visualization

In [8]:
def visualize_data(csv_path):
    df = pd.read_csv(csv_path)
    df['date'] = pd.to_datetime(df['date'])

    plt.figure(figsize=(12, 6))
    df.set_index('date').resample('M').size().plot()
    plt.title('Education Under Attack Incidents (Monthly)')
    plt.ylabel('Number of Incidents')
    plt.tight_layout()
    plt.savefig('incidents_over_time.png')

    plt.figure(figsize=(12, 6))
    df['country'].value_counts().head(10).plot(kind='bar')
    plt.title('Top 10 Countries by Incident Count')
    plt.ylabel('Number of Incidents')
    plt.tight_layout()
    plt.savefig('incidents_by_country.png')

    plt.figure(figsize=(12, 6))
    df['attack_type'].value_counts().plot(kind='pie', autopct='%1.1f%%')
    plt.title('Distribution of Attack Types')
    plt.ylabel('')
    plt.tight_layout()
    plt.savefig('attack_types.png')